In [30]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [31]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return 0.1*fpr

In [32]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [33]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [34]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [35]:
hankel = Hankel()
lag = 60
stride = 0.5

In [36]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [37]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'ATT_FLAG']

In [38]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [40]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    

In [41]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [42]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual,y_predicted))

Accuracy  0.8117647058823529
Precision  0.8
Recall  0.8695652173913043
F1-score  0.8333333333333333
False Positive Rate  0.02564102564102564


In [43]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))
print("fpr -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fpr).mean(), np.median(np.asarray(fpr)),np.asarray(fpr).min(), np.asarray(fpr).max()))

Accuracy -  Mean: 0.503967168262654 Median : 0.5058823529411764 Min : 0.4470588235294118 Max : 0.6235294117647059
precision - Mean: 0.6852574750830565 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.09049544994944388 Median : 0.08695652173913043 Min : 0.0 Max : 0.32608695652173914
f1 -        Mean: 0.15494538304208694 Median : 0.16 Min : 0.0 Max : 0.48387096774193544
fpr -        Mean: 0.000834824090638044 Median : 0.0 Min : 0.0 Max : 0.007692307692307693


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [ ]:
cluster_accuracy_1 = []
cluster_precision_1 = []
cluster_recall_1 = []
cluster_fscore_1 = []
cluster_fpr_1 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)  
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    cluster_accuracy_1.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_1.append(precision_score(y_actual,y_predicted))
    cluster_recall_1.append(recall_score(y_actual,y_predicted))
    cluster_fscore_1.append(f1_score(y_actual,y_predicted))
    cluster_fpr_1.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_1[-1])
    print("Precision ",cluster_precision_1[-1])
    print("Recall ",cluster_recall_1[-1])
    print("F1-score ",cluster_fscore_1[-1])
    print("False Positive Rate ",cluster_fpr_1[-1],"\n") 

------Number of Clusters:  2 -----------
Accuracy  0.8
Precision  0.8222222222222222
Recall  0.8043478260869565
F1-score  0.8131868131868132
False Positive Rate  0.020512820512820513 

------Number of Clusters:  3 -----------
Accuracy  0.7529411764705882
Precision  0.7450980392156863
Recall  0.8260869565217391
F1-score  0.7835051546391751
False Positive Rate  0.03333333333333333 

------Number of Clusters:  4 -----------
Accuracy  0.7647058823529411
Precision  0.7407407407407407
Recall  0.8695652173913043
F1-score  0.7999999999999999
False Positive Rate  0.0358974358974359 

------Number of Clusters:  5 -----------
Accuracy  0.7294117647058823
Precision  0.7017543859649122
Recall  0.8695652173913043
F1-score  0.7766990291262136
False Positive Rate  0.043589743589743594 

------Number of Clusters:  6 -----------
Accuracy  0.7411764705882353
Precision  0.6935483870967742
Recall  0.9347826086956522
F1-score  0.7962962962962964
False Positive Rate  0.04871794871794872 

------Number of Clu

In [ ]:
print("Accuracy Scores: ",cluster_accuracy_1)
print("Precision Scores: ",cluster_precision_1)
print("Recall Scores: ",cluster_recall_1)
print("F1 Scores: ",cluster_fscore_1)
print("False Positive Rates: ",cluster_fpr_1)

Accuracy Scores:  [0.8, 0.7529411764705882, 0.7647058823529411, 0.7294117647058823, 0.7411764705882353, 0.6823529411764706]
Precision Scores:  [0.8222222222222222, 0.7450980392156863, 0.7407407407407407, 0.7017543859649122, 0.6935483870967742, 0.6557377049180327]
Recall Scores:  [0.8043478260869565, 0.8260869565217391, 0.8695652173913043, 0.8695652173913043, 0.9347826086956522, 0.8695652173913043]
F1 Scores:  [0.8131868131868132, 0.7835051546391751, 0.7999999999999999, 0.7766990291262136, 0.7962962962962964, 0.7476635514018691]
False Positive Rates:  [0.020512820512820513, 0.03333333333333333, 0.0358974358974359, 0.043589743589743594, 0.04871794871794872, 0.05384615384615385]


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [44]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [45]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [47]:
cluster_accuracy_2 = []
cluster_precision_2 = []
cluster_recall_2 = []
cluster_fscore_2 = []
cluster_fpr_2 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_2.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_2.append(precision_score(y_actual,y_predicted))
    cluster_recall_2.append(recall_score(y_actual,y_predicted))
    cluster_fscore_2.append(f1_score(y_actual,y_predicted))
    cluster_fpr_2.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_2[-1])
    print("Precision ",cluster_precision_2[-1])
    print("Recall ",cluster_recall_2[-1])
    print("F1-score ",cluster_fscore_2[-1])
    print("False Positive Rate ",cluster_fpr_2[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.7058823529411765
Precision  0.6216216216216216
Recall  0.7931034482758621
F1-score  0.696969696969697
False Positive Rate  0.0358974358974359 /n
------Number of Clusters:  3 -----------
Accuracy  0.6764705882352942
Precision  0.6
Recall  0.7241379310344828
F1-score  0.65625
False Positive Rate  0.0358974358974359 /n
------Number of Clusters:  4 -----------
Accuracy  0.6176470588235294
Precision  0.5348837209302325
Recall  0.7931034482758621
F1-score  0.6388888888888888
False Positive Rate  0.05128205128205128 /n
------Number of Clusters:  5 -----------
Accuracy  0.6029411764705882
Precision  0.5227272727272727
Recall  0.7931034482758621
F1-score  0.6301369863013699
False Positive Rate  0.05384615384615385 /n
------Number of Clusters:  6 -----------
Accuracy  0.6176470588235294
Precision  0.5348837209302325
Recall  0.7931034482758621
F1-score  0.6388888888888888
False Positive Rate  0.05128205128205128 /n
------Number of Clusters:  7 

In [48]:
print("Accuracy Scores: ",cluster_accuracy_2)
print("Precision Scores: ",cluster_precision_2)
print("Recall Scores: ",cluster_recall_2)
print("F1 Scores: ",cluster_fscore_2)
print("False Positive Rates: ",cluster_fpr_2)

Accuracy Scores:  [0.7058823529411765, 0.6764705882352942, 0.6176470588235294, 0.6029411764705882, 0.6176470588235294, 0.5294117647058824]
Precision Scores:  [0.6216216216216216, 0.6, 0.5348837209302325, 0.5227272727272727, 0.5348837209302325, 0.4716981132075472]
Recall Scores:  [0.7931034482758621, 0.7241379310344828, 0.7931034482758621, 0.7931034482758621, 0.7931034482758621, 0.8620689655172413]
F1 Scores:  [0.696969696969697, 0.65625, 0.6388888888888888, 0.6301369863013699, 0.6388888888888888, 0.6097560975609757]
False Positive Rates:  [0.0358974358974359, 0.0358974358974359, 0.05128205128205128, 0.05384615384615385, 0.05128205128205128, 0.0717948717948718]


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [50]:
cluster_accuracy_3 = []
cluster_precision_3 = []
cluster_recall_3 = []
cluster_fscore_3 = []
cluster_fpr_3 = []
for k in range(2,10):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    sensor_mean = np.mean(sensor_predicted, axis=0)
    y_predicted = (sensor_mean > 0.1).astype(int)
    # y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_3.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_3.append(precision_score(y_actual,y_predicted))
    cluster_recall_3.append(recall_score(y_actual,y_predicted))
    cluster_fscore_3.append(f1_score(y_actual,y_predicted))
    cluster_fpr_3.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_3[-1])
    print("Precision ",cluster_precision_3[-1])
    print("Recall ",cluster_recall_3[-1])
    print("F1-score ",cluster_fscore_3[-1])
    print("False Positive Rate ",cluster_fpr_3[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.6176470588235294
Precision  0.5333333333333333
Recall  0.8275862068965517
F1-score  0.6486486486486487
False Positive Rate  0.05384615384615385 /n
------Number of Clusters:  3 -----------
Accuracy  0.6029411764705882
Precision  0.5294117647058824
Recall  0.6206896551724138
F1-score  0.5714285714285715
False Positive Rate  0.041025641025641026 /n
------Number of Clusters:  4 -----------
Accuracy  0.4117647058823529
Precision  0.417910447761194
Recall  0.9655172413793104
F1-score  0.5833333333333333
False Positive Rate  0.1 /n
------Number of Clusters:  5 -----------
Accuracy  0.6029411764705882
Precision  0.52
Recall  0.896551724137931
F1-score  0.6582278481012658
False Positive Rate  0.06153846153846154 /n
------Number of Clusters:  6 -----------
Accuracy  0.5441176470588235
Precision  0.48214285714285715
Recall  0.9310344827586207
F1-score  0.6352941176470589
False Positive Rate  0.07435897435897436 /n
------Number of Clusters:  7 -

In [51]:
print("Accuracy Scores: ",cluster_accuracy_3)
print("Precision Scores: ",cluster_precision_3)
print("Recall Scores: ",cluster_recall_3)
print("F1 Scores: ",cluster_fscore_3)
print("False Positive Rates: ",cluster_fpr_3)

Accuracy Scores:  [0.6176470588235294, 0.6029411764705882, 0.4117647058823529, 0.6029411764705882, 0.5441176470588235, 0.4264705882352941, 0.5588235294117647, 0.5147058823529411]
Precision Scores:  [0.5333333333333333, 0.5294117647058824, 0.417910447761194, 0.52, 0.48214285714285715, 0.4264705882352941, 0.4897959183673469, 0.46774193548387094]
Recall Scores:  [0.8275862068965517, 0.6206896551724138, 0.9655172413793104, 0.896551724137931, 0.9310344827586207, 1.0, 0.8275862068965517, 1.0]
F1 Scores:  [0.6486486486486487, 0.5714285714285715, 0.5833333333333333, 0.6582278481012658, 0.6352941176470589, 0.5979381443298969, 0.6153846153846154, 0.6373626373626373]
False Positive Rates:  [0.05384615384615385, 0.041025641025641026, 0.1, 0.06153846153846154, 0.07435897435897436, 0.1, 0.06410256410256411, 0.08461538461538462]
